In [3]:
from experiment_project.utils.initial.util import init_sys_env
from experiment_project.utils.files.read import read_yaml
import dspy
init_sys_env()
secret_env_file = '/mnt/d/project/dy/extra/autogen/env_secret_config.yaml'

api_configs = read_yaml(secret_env_file)
model_config = api_configs.get('openai')
turbo = dspy.OpenAI(model=model_config.get('model'), max_tokens=1024,api_key=model_config.get('api_key'))
dspy.settings.configure(lm=turbo)

In [2]:
import dspy

# 定义 Signature
class BasicQA(dspy.Signature):
    """回答问题，答案要简短。"""
    question = dspy.InputField()
    answer = dspy.OutputField(desc="通常是 1 到 25 个词", prefix="问题的答案:")

# 创建任务模块
class QAModule(dspy.Module):
    def __init__(self):
        super().__init__()
        # 使用 BasicQA Signature 定义任务
        self.qa_task = dspy.Predict(BasicQA)

    def forward(self, question):
        # 执行任务并返回结果
        return self.qa_task(question=question)


# 创建 QAModule 实例
qa_module = QAModule()
# 定义问题
question = "成都最好吃的食物是什么？"
# 执行任务并获取答案
response = qa_module.forward(question)
print(response.answer)  # 输出答案，例如 "问题的答案: 珠穆朗玛峰"
turbo.inspect_history(n=1)
# 查看历史记录


In [5]:
turbo.history[0]['prompt']



In [30]:

# 创建任务模块
class QAModule(dspy.Module):
    def __init__(self):
        super().__init__()
        self.cot = dspy.ChainOfThought('question -> answer',n=5)
        # 使用 Predict 获取最终答案
    def forward(self, question):

        return self.cot(question= question)

# 创建 MathSolver 实例
qa_module = QAModule()

# 定义问题
problem = """
德国二战为什么战败? 使用中文回答
"""
# 解决问题
response = qa_module.forward(question=problem)
response
# 打印结果


In [31]:
turbo.history[-1]

## Retrieve 学习

In [27]:


import dspy

# 1. 选择 "搜索引擎":  这里使用 ColBERTv2 模型

colbert = dspy.ColBERTv2(url='http://20.102.90.50:2017/wiki17_abstracts') 
dspy.settings.configure(rm=colbert)  # 配置 DSPy 使用这个模型

# 2. 创建 Retrieve 对象,  指定返回 3 个结果
retriever = dspy.Retrieve(k=4) 

# 3.  输入问题，获取答案
question = "openai是什么公司？"
results = retriever(question).passages  # 获取相关段落

# 打印结果
for i, passage in enumerate(results):
    print(f"{i+1}. {passage}") 


In [37]:
import dspy



# 定义 Signature: 输入是两个字符串表示的数字，输出是它们的和
class AddTwoNumbers(dspy.Signature):
    a = dspy.InputField(desc="第一个数字", type=str)
    b = dspy.InputField(desc="第二个数字", type=str)
    sum = dspy.OutputField(desc="两个数字的和", type=str)

# 创建 PoT 对象
pot = dspy.ProgramOfThought(AddTwoNumbers)

# 定义问题，将数字转换为字符串
problem = {"a": str(10), "b": str(20)}  

# 解决问题
response = pot(**problem)

# 打印结果
print(response.sum)  # 输出: 30


In [38]:
turbo.history[-1]

In [40]:

# 定义 Signature
class DayOfWeek(dspy.Signature):
    today = dspy.InputField(desc="今天是星期几", type=str)
    tomorrow = dspy.OutputField(desc="明天是星期几", type=str)

# 创建 CoT 对象
cot = dspy.ChainOfThought(DayOfWeek)

# 定义问题
problem = {"today": "星期一"}

# 解决问题
response = cot(**problem)

# # 打印 Chain of Thought 推理过程和结果
# print("Chain of Thought 推理过程:")
# for step in response.thoughts:
#     print(step)
# 
# print(f"最终答案: 明天是 {response.output.tomorrow}")


In [43]:
response

In [7]:
import dspy
# 定义一个用于基本问答的简单签名
class BasicQA(dspy.Signature):
    question = dspy.InputField()
    answer = dspy.OutputField(desc="经常在1到5个单词之间")

# Pass signature to ReAct module
react_module = dspy.ReAct(BasicQA,)

# Call the ReAct module on a particular input
question = '天空的颜色是什么？使用中文回答!'
result = react_module(question=question)

print(f"Question: {question}")
print(f"最终预测答案（经过ReAct过程后）: {result.answer}")

In [9]:
turbo.inspect_history(n=3)

In [6]:
"""
你将获得 `问题` 并且你将用 `答案` 回答。为此，你将交错进行思考、行动和观察步骤。思考可以推理当前情况，行动可以是以下类型：（1）搜索[查询]，这需要一个搜索查询并从语料库中返回一个或多个潜在相关的段落（2）完成[答案]，这返回最终的 `答案` 并完成任务——遵循以下格式。问题: {问题} 思考 1: 根据上次观察采取的下一步行动 行动 1: 总是搜索[查询]，或者在完成时，完成[答案] —— 问题: 天空的颜色是什么？ 思考 1: 天空的颜色可以因一天中的时间和天气状况而异，但通常在晴朗的白天被认为是蓝色的。 行动 1: 完成 [在晴朗的白天，天空的颜色通常是蓝色的。]
"""

In [12]:
  # ChainOfThought example
# 创建任务模块  
class MathQuestion(dspy.Module):  
    def __init__(self):  
        super().__init__()  
        # 使用 ChainOfThought 进行逐步推理  
        self.think = dspy.ChainOfThought('question -> answer')  
    def forward(self, question:str):    
        # 组合完整解决方案  
        return self.think(question= question)   
  
# 创建 MathSolver 实例  
math_question = MathQuestion()  
  
# 定义问题  
question = """  
现在笼子里有鸡（雉）和兔子在一起。从上面数一共有三十五个头，从下面数一共有九十四只脚，问一共有多少只鸡、多少只兔子？
使用中文回答 
"""  
  
# 解决问题  
response = math_question.forward(question=question)  
  
response.answer

In [13]:
turbo.inspect_history(n=10)